In [1]:
import gc
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GroupKFold, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.metrics import precision_recall_curve, auc

from collections import defaultdict
from tqdm import tqdm

In [ ]:
MAIN_PATH = 'avito-train-combined/data/avito-merged-dataset/'
ECOM_PRETRAIN = 'avito-train-combined/data/clip-marqofashionsiglip-marqoecom-top2kaggle/'
RESNET_PATH = 'avito-train-combined/data/resnet-cossim/'
KAGGLE_TOP5 = 'avito-train-combined/data/top5-kaggle/'
BERTA_PATH = 'avito-train-combined/data/berta-pretrained-cossims/'
RUBERT_TINY_OOF_PATH = 'avito-train-combined/data/rubert-folds/'
RUBERT_TINY_PREDS_PATH = 'avito-train-combined/data/rubert-test-preds/'
E5LARGE_OOF_PATH = 'avito-train-combined/data/avito-e5-large-pretrain/'
E5LARGE_PREDS_PATH = 'avito-train-combined/data/avito-e5-large-test/'
REV_RUBERT_TINY_OOF_PATH = 'avito-train-combined/data/name_desc_bert_oof_rev/'
REV_RUBERT_TINY_PREDS_PATH = 'avito-train-combined/data/name-desc-bert-preds-rev/'
USERBGE_COSSIMS_PATH = 'avito-train-combined/data/userbge-cossims/'
RUBERT_BASE_TEST_PREDS_PATH = 'avito-train-combined/data/rubert-fixed-test-preds/'
RUBERT_BASE_TEST_PREDS_REV_PATH = 'avito-train-combined/data/rubert-fixed-test-preds-rev/'
FT_PREDS_PATH = 'avito-train-combined/data/ft-preds2/'
RUBERT_BASE_OOF_PATH = 'avito-train-combined/data/trained-rubert-base-preds/'
RUBERT_BASE_OOF_REV_PATH = 'avito-train-combined/data/trained-rubert-base-preds-rev/'

USE_MEAN_BASE_AND_REV = False

In [3]:
train = pd.read_parquet(MAIN_PATH + 'train_df.parquet')
test = pd.read_parquet(MAIN_PATH + 'test_df.parquet')

In [4]:
to_drop = [
    'category_level_1_1', 'category_level_1_2',
    'category_level_2_1', 'category_level_2_2',
    'category_level_3_1', 'category_level_3_2',
    'category_level_4_1', 'category_level_4_2',
]

train[to_drop] = train[to_drop].fillna('none')

train['unique_cat_1'] = train['category_level_1_1'] + '_' + train['category_level_1_2']
train['unique_cat_2'] = train['category_level_2_1'] + '_' + train['category_level_2_2']
train['unique_cat_3'] = train['category_level_3_1'] + '_' + train['category_level_3_2']
train['unique_cat_4'] = train['category_level_4_1'] + '_' + train['category_level_4_2']

test[to_drop] = test[to_drop].fillna('none')

test['unique_cat_1'] = test['category_level_1_1'] + '_' + test['category_level_1_2']
test['unique_cat_2'] = test['category_level_2_1'] + '_' + test['category_level_2_2']
test['unique_cat_3'] = test['category_level_3_1'] + '_' + test['category_level_3_2']
test['unique_cat_4'] = test['category_level_4_1'] + '_' + test['category_level_4_2']

train.drop(columns=to_drop, axis=1, inplace=True)
gc.collect()

test.drop(columns=to_drop, axis=1, inplace=True)
gc.collect()

0

In [5]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                    gc.collect()
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                    gc.collect()
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                    gc.collect()
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
                    gc.collect()
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                    gc.collect()
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                    gc.collect()
                else:
                    df[col] = df[col].astype(np.float64)
                    gc.collect()
        else:
            if df[col].nunique() == 2:
                df[col] = df[col].astype('bool')
            gc.collect()

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    gc.collect()
    
    return df

In [6]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 6521.05 MB
Memory usage after optimization is: 1842.67 MB
Decreased by 71.7%


In [7]:
test = reduce_mem_usage(test)

Memory usage of dataframe is 1728.06 MB
Memory usage after optimization is: 490.67 MB
Decreased by 71.6%


In [8]:
train = train.sort_values(by=['variantid_1', 'variantid_2'])
test = test.sort_values(by=['variantid_1', 'variantid_2'])

In [ ]:
# --- IMG FEATURES ---

# pretrain clip
train_clip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_train_CLIP.parquet')
test_clip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_test_CLIP.parquet')

train_clip = train_clip.sort_values(by=['variantid_1', 'variantid_2'])
test_clip = test_clip.sort_values(by=['variantid_1', 'variantid_2'])
train['clip_cosine_sim'] = train_clip['cosine_sim']
test['clip_cosine_sim'] = test_clip['cosine_sim']
del train_clip, test_clip

# pretrain fashion siglip
train_fashionsiglip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_fashion_clip_train.parquet')
test_fashionsiglip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_fashion_clip_test.parquet')

train_fashionsiglip = train_fashionsiglip.sort_values(by=['variantid_1', 'variantid_2'])
test_fashionsiglip = test_fashionsiglip.sort_values(by=['variantid_1', 'variantid_2'])
train['fashionsiglip_cosine_sim'] = train_fashionsiglip['cosine_sim']
test['fashionsiglip_cosine_sim'] = test_fashionsiglip['cosine_sim']
del train_fashionsiglip, test_fashionsiglip

# pretrain marqo ecom
train_ecom = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_ecomm_train.parquet')
test_ecom = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_ecomm_test.parquet')

train_ecom = train_ecom.sort_values(by=['variantid_1', 'variantid_2'])
test_ecom = test_ecom.sort_values(by=['variantid_1', 'variantid_2'])
train['ecom_cosine_sim'] = train_ecom['cosine_sim']
test['ecom_cosine_sim'] = test_ecom['cosine_sim']
del train_ecom, test_ecom

# kaggle top2 model, bugged
# train_top2kaggle = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_top2_kaggle_train.parquet')
# test_top2kaggle = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_top2_kaggle_test.parquet')

# train_top2kaggle = train_top2kaggle.sort_values(by=['variantid_1', 'variantid_2'])
# test_top2kaggle = test_top2kaggle.sort_values(by=['variantid_1', 'variantid_2'])
# train['top2kaggle_cosine_sim'] = train_top2kaggle['cosine_sim']
# test['top2kaggle_cosine_sim'] = test_top2kaggle['cosine_sim']
# del train_top2kaggle, test_top2kaggle

# kaggle top5 model
train_top5kaggle = pd.read_parquet(KAGGLE_TOP5 + 'cossim_final_concat_train.parquet')
test_top5kaggle = pd.read_parquet(KAGGLE_TOP5 + 'cossim_final_concat_test.parquet')

train_top5kaggle = train_top5kaggle.sort_values(by=['variantid_1', 'variantid_2'])
test_top5kaggle = test_top5kaggle.sort_values(by=['variantid_1', 'variantid_2'])
train['top5kaggle_cosine_sim'] = train_top5kaggle['cosine_sim']
test['top5kaggle_cosine_sim'] = test_top5kaggle['cosine_sim']
del train_top5kaggle, test_top5kaggle

# trained resnet
train_resnet = pd.read_parquet(RESNET_PATH + 'train_resnet_cossim.parquet')
test_resnet = pd.read_parquet(RESNET_PATH + 'test_resnet_cossim.parquet')

train_resnet = train_resnet.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test_resnet = test_resnet.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
train['resnet_cosine_sim'] = train_resnet['cossims_resnet']
test['resnet_cosine_sim'] = test_resnet['cossims_resnet']
del train_resnet, test_resnet

gc.collect()

# --- TEXT FEATURES ---

# pretrain berta
berta_cossims_train_part1 = pd.read_parquet(BERTA_PATH + 'berta_cossims_train_part1.parquet')
berta_cossims_train_part2 = pd.read_parquet(BERTA_PATH + 'berta_cossims_train_part2.parquet')
berta_cossims_train = pd.concat([berta_cossims_train_part1, berta_cossims_train_part2])
berta_cossims_test = pd.read_parquet(BERTA_PATH + 'berta_cossims_test.parquet')

berta_cossims_train = berta_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])
berta_cossims_test = berta_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])
train['berta_cossim'] = berta_cossims_train['berta_cossim']
test['berta_cossim'] = berta_cossims_test['berta_cossim']

del berta_cossims_train_part1, berta_cossims_train_part2, berta_cossims_train, berta_cossims_test
gc.collect()

# trained rubert (cherez zhopu rukamu obuchen)
rubert_oof_fold0 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold0.parquet')
rubert_oof_fold1 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold1.parquet')
rubert_oof_fold2 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold2.parquet')
rubert_oof_fold3 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold3.parquet')
rubert_oof_fold4 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold4.parquet')

rubert_test_pred_fold0 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_0.parquet')
rubert_test_pred_fold1 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_1.parquet')
rubert_test_pred_fold2 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_2.parquet')
rubert_test_pred_fold3 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_3.parquet')
rubert_test_pred_fold4 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_4.parquet')

rubert_oof_fold0.rename(columns={'name_desc_bert_oof1': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold1.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold2.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold3.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold4.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)

rubert_oof_fold0 = rubert_oof_fold0.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold1 = rubert_oof_fold1.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold2 = rubert_oof_fold2.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold3 = rubert_oof_fold3.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold4 = rubert_oof_fold4.sort_values(by=['variantid_1', 'variantid_2'])

rubert_oof = rubert_oof_fold0['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold1['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold2['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold3['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold4['name_desc_rubert_tiny_turbo_2048_wce']

rubert_test_pred_fold0 = rubert_test_pred_fold0.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold1 = rubert_test_pred_fold1.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold2 = rubert_test_pred_fold2.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold3 = rubert_test_pred_fold3.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold4 = rubert_test_pred_fold4.sort_values(by=['variantid_1', 'variantid_2'])

rubert_preds = (rubert_test_pred_fold0['name_desc_rubert_tiny_turbo_2048_wce_0'] + rubert_test_pred_fold1['name_desc_rubert_tiny_turbo_2048_wce_1'] + rubert_test_pred_fold2['name_desc_rubert_tiny_turbo_2048_wce_2'] + rubert_test_pred_fold3['name_desc_rubert_tiny_turbo_2048_wce_3'] + rubert_test_pred_fold4['name_desc_rubert_tiny_turbo_2048_wce_4']) / 5

train['name_desc_rubert_tiny_turbo_2048_wce'] = rubert_oof
test['name_desc_rubert_tiny_turbo_2048_wce'] = rubert_preds

del rubert_oof_fold0, rubert_oof_fold1, rubert_oof_fold2, rubert_oof_fold3, rubert_oof_fold4
del rubert_oof
del rubert_test_pred_fold0, rubert_test_pred_fold1, rubert_test_pred_fold2, rubert_test_pred_fold3, rubert_test_pred_fold4
del rubert_preds
gc.collect()

# pretrain e5large
e5large_cossims_train_part1 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold0.parquet')
e5large_cossims_train_part2 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold1.parquet')
e5large_cossims_train_part3 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold2.parquet')
e5large_cossims_train_part4 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold3.parquet')
e5large_cossims_train_part5 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold4.parquet')

e5large_cossims_train = pd.concat([
    e5large_cossims_train_part1,
    e5large_cossims_train_part2,
    e5large_cossims_train_part3,
    e5large_cossims_train_part4,
    e5large_cossims_train_part5
])
e5large_cossims_train = e5large_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])

del e5large_cossims_train_part1, e5large_cossims_train_part2, e5large_cossims_train_part3, e5large_cossims_train_part4, e5large_cossims_train_part5
gc.collect()

e5large_cossims_test_part1 = pd.read_parquet(E5LARGE_PREDS_PATH + 'e5large_cossims_part1.parquet')
e5large_cossims_test_part2 = pd.read_parquet(E5LARGE_PREDS_PATH + 'e5large_cossims_part2.parquet')

e5large_cossims_test = pd.concat([
    e5large_cossims_test_part1,
    e5large_cossims_test_part2,
])
e5large_cossims_test = e5large_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])

del e5large_cossims_test_part1, e5large_cossims_test_part2
gc.collect()

train['e5large_cossim'] = e5large_cossims_train['e5large_cossim']
test['e5large_cossim'] = e5large_cossims_test['e5large_cossim']

del e5large_cossims_train, e5large_cossims_test
gc.collect()

# rev trained rubert (cherez zhopu rukamu obuchen)
rubert_oof_rev = pd.read_parquet(REV_RUBERT_TINY_OOF_PATH + 'name_desc_bert_oof_rev.parquet')
rubert_preds_rev = pd.read_parquet(REV_RUBERT_TINY_PREDS_PATH + 'name_desc_bert_preds_rev.parquet')

rubert_oof_rev = rubert_oof_rev.sort_values(by=['variantid_1', 'variantid_2'])
rubert_preds_rev = rubert_preds_rev.sort_values(by=['variantid_1', 'variantid_2'])
train['name_desc_rubert_tiny_turbo_2048_wce_rev'] = rubert_oof_rev['name_desc_bert_oof_rev']
test['name_desc_rubert_tiny_turbo_2048_wce_rev'] = rubert_preds_rev['name_desc_bert_preds_rev']
del rubert_oof_rev, rubert_preds_rev

gc.collect()

# rubert tta
if USE_MEAN_BASE_AND_REV:
    train['name_desc_rubert_tiny_turbo_2048_wce_tta'] = (train['name_desc_rubert_tiny_turbo_2048_wce'] + train['name_desc_rubert_tiny_turbo_2048_wce_rev']) / 2
    test['name_desc_rubert_tiny_turbo_2048_wce_tta'] = (test['name_desc_rubert_tiny_turbo_2048_wce'] + test['name_desc_rubert_tiny_turbo_2048_wce_rev']) / 2
    del train['name_desc_rubert_tiny_turbo_2048_wce'], train['name_desc_rubert_tiny_turbo_2048_wce_rev']
    del test['name_desc_rubert_tiny_turbo_2048_wce'], test['name_desc_rubert_tiny_turbo_2048_wce_rev']
    gc.collect()

# pretrain userbge
userbge_cossims_train = pd.DataFrame()
userbge_cossims_test = pd.DataFrame()

for i in range(1, 16):
    curr_df = pd.read_parquet(USERBGE_COSSIMS_PATH + f'userbge_cossims_train_part{i}.parquet')
    userbge_cossims_train = pd.concat([userbge_cossims_train, curr_df])
    print(f'{userbge_cossims_train.shape=}')
userbge_cossims_train = userbge_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])

for i in range(1, 5):
    curr_df = pd.read_parquet(USERBGE_COSSIMS_PATH + f'/userbge_cossims_test_part{i}.parquet')
    userbge_cossims_test = pd.concat([userbge_cossims_test, curr_df])
    print(f'{userbge_cossims_test.shape=}')
userbge_cossims_test = userbge_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])

del curr_df
gc.collect()

train['userbge_cossim'] = userbge_cossims_train['userbge_cossim']
test['userbge_cossim'] = userbge_cossims_test['userbge_cossim']

del userbge_cossims_train, userbge_cossims_test
gc.collect()

# --- тут именно до сэмпла! ---

# trained rubert-base test preds
rubert_base_pred = pd.read_csv(
    RUBERT_BASE_TEST_PREDS_PATH + 'rubert_ZAEBAL_SUKA.csv'
).rename(columns={'base_id': 'variantid_1', 'cand_id': 'variantid_2'})
rubert_base_pred = rubert_base_pred.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test['rubert_base_trained'] = rubert_base_pred['probability']

rubert_base_pred_rev = pd.read_csv(
    RUBERT_BASE_TEST_PREDS_REV_PATH + 'rubert_ZAEBAL_SUKA_REV.csv'
).rename(columns={'base_id': 'variantid_1', 'cand_id': 'variantid_2'})
rubert_base_pred_rev = rubert_base_pred_rev.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test['rubert_base_trained_rev'] = rubert_base_pred_rev['probability']

del rubert_base_pred, rubert_base_pred_rev
gc.collect()

# rubert base test tta
if USE_MEAN_BASE_AND_REV:
    test['rubert_base_trained_tta'] = (test['rubert_base_trained'] + test['rubert_base_trained_rev']) / 2
    del test['rubert_base_trained'],  test['rubert_base_trained_rev']
    gc.collect()

# --- sample!!! ---
train = train.sample(len(train), random_state=42)

# trained ft (именно после сэмпла! тут уже все карты в нужном порядке разложены)
fasttext_train = joblib.load(FT_PREDS_PATH + 'oof_preds.pkl')
fasttext_test = joblib.load(FT_PREDS_PATH + 'test_preds.pkl')
fasttext_train_rev = joblib.load(FT_PREDS_PATH + 'oof_preds_rev.pkl')
fasttext_test_rev = joblib.load(FT_PREDS_PATH + 'test_preds_rev.pkl')

train['fasttext'] = fasttext_train
test['fasttext'] = fasttext_test
train['fasttext_rev'] = fasttext_train_rev
test['fasttext_rev'] = fasttext_test_rev

del fasttext_train, fasttext_test, fasttext_train_rev, fasttext_test_rev
gc.collect()

# fasttext tta
if USE_MEAN_BASE_AND_REV:
    train['fasttext_tta'] = (train['fasttext'] + train['fasttext_rev']) / 2
    test['fasttext_tta'] = (test['fasttext'] + test['fasttext_rev']) / 2
    del train['fasttext'], train['fasttext_rev']
    del test['fasttext'], test['fasttext_rev']
    gc.collect()

# trained rubert-base train (именно после сэмпла! тут уже все карты в нужном порядке разложены)
rubert_base_oof = joblib.load(RUBERT_BASE_OOF_PATH + 'oof_preds_rubert_base.joblib')
train['rubert_base_trained'] = rubert_base_oof

rubert_base_oof_rev = joblib.load(RUBERT_BASE_OOF_REV_PATH + 'oof_preds_rubert_base_rev.joblib')
train['rubert_base_trained_rev'] = rubert_base_oof_rev

del rubert_base_oof, rubert_base_oof_rev
gc.collect()

# rubert train test tta
if USE_MEAN_BASE_AND_REV:
    train['rubert_base_trained_tta'] = (train['rubert_base_trained'] + train['rubert_base_trained_rev']) / 2
    del train['rubert_base_trained'],  train['rubert_base_trained_rev']
    gc.collect()

# add rouge features (w/ tta)

train_rouge = pd.read_csv('rouge-avito/train_rouge.csv')
test_rouge = pd.read_csv('/rouge-avito/test_rouge.csv')

train_rouge = train_rouge.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test_rouge = test_rouge.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)

train['rouge_1'] = train_rouge['rouge_1']
train['rouge_2'] = train_rouge['rouge_2']
train['rouge_3'] = train_rouge['rouge_3']
train['rouge_4'] = train_rouge['rouge_4']
train['rouge_s4'] = train_rouge['rouge_s4']
train['rouge_su4'] = train_rouge['rouge_su4']

test['rouge_1'] = test_rouge['rouge_1']
test['rouge_2'] = test_rouge['rouge_2']
test['rouge_3'] = test_rouge['rouge_3']
test['rouge_4'] = test_rouge['rouge_4']
test['rouge_s4'] = test_rouge['rouge_s4']
test['rouge_su4'] = test_rouge['rouge_su4']

userbge_cossims_train.shape=(125303, 3)
userbge_cossims_train.shape=(250606, 3)
userbge_cossims_train.shape=(375909, 3)
userbge_cossims_train.shape=(501212, 3)
userbge_cossims_train.shape=(626515, 3)
userbge_cossims_train.shape=(751818, 3)
userbge_cossims_train.shape=(877121, 3)
userbge_cossims_train.shape=(1002424, 3)
userbge_cossims_train.shape=(1127727, 3)
userbge_cossims_train.shape=(1253030, 3)
userbge_cossims_train.shape=(1378333, 3)
userbge_cossims_train.shape=(1503636, 3)
userbge_cossims_train.shape=(1628939, 3)
userbge_cossims_train.shape=(1754242, 3)
userbge_cossims_train.shape=(1879555, 3)
userbge_cossims_test.shape=(125000, 3)
userbge_cossims_test.shape=(250000, 3)
userbge_cossims_test.shape=(375000, 3)
userbge_cossims_test.shape=(500000, 3)


In [10]:
# pl_test = test.copy()
# pl_test = pl_test.sort_values(by=['variantid_1', 'variantid_2'])

# top_submit = pd.read_csv('/kaggle/input/top-sub-19-05/zhiv_el_trans_coef_1_25_models_0_5baselama_0_5(0_3catlamas_0_7catcbs)(1).csv')
# top_submit = top_submit.sort_values(by=['base_id', 'cand_id'])

# pl_test = pl_test.reset_index(drop=True)
# top_submit = top_submit.reset_index(drop=True)

# pl_test['is_double'] = top_submit['probability']

# pl_test = pl_test[pl_test['unique_cat_1'].isin(['Услуги_Услуги', 'Недвижимость_Недвижимость', 'Работа_Работа'])]

# pl_test = pl_test[~pl_test['is_double'].between(0.02, 0.98)]
# pl_test['is_double'] = np.where(
#     pl_test['is_double'] < 0.02, 0, 
#     np.where(pl_test['is_double'] > 0.98, 1, pl_test['is_double'])
# )
# pl_test['group_id'] = 777777

# train = pd.concat([train, pl_test], axis=0).reset_index(drop=True)

In [11]:
train.drop(columns=['variantid_1', 'variantid_2', 'base_title_image', 'cand_title_image'], axis=1, inplace=True)
test.drop(columns=['variantid_1', 'variantid_2', 'base_title_image', 'cand_title_image'], axis=1, inplace=True)
train.drop(columns=['action_date'], inplace=True)

In [12]:
cat_features = [
    'is_same_location',
    'is_same_region',
    'category_level_1_match',
    'category_level_2_match',
    'category_level_3_match',
    'category_level_4_match',
    'category_level_3_fillness',
    'category_level_4_fillness',
    'n_images_fillness',
    'unique_cat_1',
    'unique_cat_2',
    'unique_cat_3',
    'unique_cat_4',
]

train[cat_features] = train[cat_features].astype(str)
test[cat_features] = test[cat_features].astype(str)

In [13]:
# cat_mapping = {
#     '1.0': 'True',
#     '0.0': 'False'
# }

In [14]:
# test['category_level_1_match'] = test['category_level_1_match'].map(cat_mapping)
# test['category_level_2_match'] = test['category_level_2_match'].map(cat_mapping)

In [15]:
features = [col for col in train.columns if col not in ['group_id', 'is_double']]
target = 'is_double'

In [16]:
gc.collect()

0

In [17]:
len(features)

470

In [18]:
# # обычный кб

# gkf = StratifiedGroupKFold(n_splits=5) # GroupKFold(n_splits=5)

# oof_preds_cb = np.zeros(len(train))

# for fold, (train_idx, val_idx) in enumerate(gkf.split(train, train[target], groups=train['group_id'])):
#     print(f"fold {fold+1}...")
    
#     X_train, X_val = train.iloc[train_idx][features], train.iloc[val_idx][features]
#     y_train, y_val = train.iloc[train_idx][target], train.iloc[val_idx][target]

#     model_cb = CatBoostClassifier(
#         iterations=50000,
#         learning_rate=0.1,
#         cat_features=cat_features,
#         # auto_class_weights='Balanced',
#         # loss_function='Focal:focal_alpha=0.75;focal_gamma=2',
#         eval_metric='PRAUC',
#         random_seed=42+fold,
#         early_stopping_rounds=2500,
#         task_type='GPU',
#         verbose=100
#     )
    
#     model_cb.fit(X_train, y_train, eval_set=(X_val, y_val))
#     oof_preds_cb[val_idx] = model_cb.predict_proba(X_val)[:, 1]

#     joblib.dump(model_cb, f"catboost_model_fold_{fold+1}.joblib")

#     del model_cb, X_train, X_val, y_train, y_val
#     gc.collect()

# joblib.dump(oof_preds_cb, "oof_preds_cb_5folds.joblib")

In [19]:
# обычный кб но 5 сидов

# gkf = StratifiedGroupKFold(n_splits=5)
# oof_preds_cb = np.zeros(len(train))

# for fold, (train_idx, val_idx) in enumerate(gkf.split(train, train[target], groups=train['group_id'])):
#     print(f"Fold {fold+1}...")
    
#     X_train, X_val = train.iloc[train_idx][features], train.iloc[val_idx][features]
#     y_train, y_val = train.iloc[train_idx][target], train.iloc[val_idx][target]
    
#     fold_val_preds = np.zeros(len(X_val))

#     for seed in range(5):
#         print(f"  Seed {seed+1}...")
#         model_cb = CatBoostClassifier(
#             iterations=50000,
#             learning_rate=0.1,
#             cat_features=cat_features,
#             eval_metric='PRAUC',
#             random_seed=42+fold*10+seed, 
#             early_stopping_rounds=2500,
#             task_type='GPU',
#             verbose=100
#         )
        
#         model_cb.fit(X_train, y_train, eval_set=(X_val, y_val))
#         fold_val_preds += model_cb.predict_proba(X_val)[:, 1]
        
#         joblib.dump(model_cb, f"catboost_model_fold_{fold+1}_seed_{seed+1}.joblib")
        
#         del model_cb
#         gc.collect()
    
#     oof_preds_cb[val_idx] = fold_val_preds / 5

# joblib.dump(oof_preds_cb, "oof_preds_cb_5folds_5seeds.joblib")

In [20]:
# precision, recall, thresholds = precision_recall_curve(train[target], oof_preds_cb)
# oof_prauc_cb = auc(recall, precision)
# oof_prauc_cb

In [21]:
def predict_ensemble(test_data, models, features):
    predictions = np.zeros((len(test_data), len(models)))
    
    for i, model in tqdm(enumerate(models)):
        predictions[:, i] = model.predict_proba(test_data[features])[:, 1]
    
    ensemble_pred = predictions.mean(axis=1)
    
    return ensemble_pred

In [22]:
# models_cb = [joblib.load(f'/kaggle/working/catboost_model_fold_{fold+1}.joblib') for fold in range(5)]
# models_cb = []
# for fold in range(5):
#     for seed in range(5):
#         model_path = f'/kaggle/input/catboost-123-models/catboost_model_fold_{fold+1}_seed_{seed+1}.joblib'
#         models_cb.append(joblib.load(model_path))

In [23]:
# ensemble_predictions_cb = predict_ensemble(test, models_cb, features)

In [24]:
# joblib.dump(ensemble_predictions_cb, f"ensemble_predictions_cb.joblib")

In [25]:
# fig, axes = plt.subplots(5, 1, figsize=(12, 36))

# for i, model in enumerate(models_cb):
#     feature_importances = model.get_feature_importance()
#     feature_names = features
#     sorted_idx = np.argsort(feature_importances)[::-1][:50]
#     top_features = np.array(feature_names)[sorted_idx]
#     top_importances = feature_importances[sorted_idx]
    
#     axes[i].barh(top_features[::-1], top_importances[::-1])
#     axes[i].set_title(f'Feature Importance Top 50 - Fold {i+1}')
#     axes[i].set_xlabel('Importance')
#     axes[i].set_ylabel('Feature')

# plt.tight_layout()
# plt.show()

In [26]:
# кб по категориям 1 уровня

categories = train['unique_cat_1'].unique()
categories = [cat.split('_')[0] for cat in categories if cat.split('_')[0] == cat.split('_')[1]]

gkf = StratifiedGroupKFold(n_splits=5) # GroupKFold(n_splits=5)

cat_oof_preds_cb = {}
oof_metrics_cat = {}

progress_bar = tqdm(categories)

for category in progress_bar:
    progress_bar.set_description(f"{category=}")

    train_category = train[train['unique_cat_1'] == f'{category}_{category}']
    test_category = test[test['unique_cat_1'] == f'{category}_{category}']

    cat_oof_preds_cb[category] = np.zeros(len(train_category))

    for fold, (train_idx, val_idx) in enumerate(gkf.split(train_category, train_category[target], groups=train_category['group_id'])):
        print(f"fold {fold+1}...")
    
        X_train_cat, X_val_cat = train_category.iloc[train_idx][features], train_category.iloc[val_idx][features]
        y_train_cat, y_val_cat = train_category.iloc[train_idx][target], train_category.iloc[val_idx][target]

        model_fold_cb = CatBoostClassifier(
            iterations=50000,
            # learning_rate=0.1,
            cat_features=cat_features,
            # auto_class_weights='Balanced',
            eval_metric='PRAUC',
            random_seed=69+fold,
            early_stopping_rounds=1000,
            task_type='GPU',
            verbose=100
        )

        model_fold_cb.fit(X_train_cat, y_train_cat, eval_set=(X_val_cat, y_val_cat))
        cat_oof_preds_cb[category][val_idx] = model_fold_cb.predict_proba(X_val_cat)[:, 1]

        joblib.dump(model_fold_cb, f"{category}_catboost_model_fold_{fold+1}.joblib")

        del model_fold_cb
        gc.collect()

    precision, recall, thresholds = precision_recall_curve(train_category[target], cat_oof_preds_cb[category])
    cat_oof_prauc_cb = auc(recall, precision)
    print(f'{cat_oof_prauc_cb=}')

    oof_metrics_cat[category] = {'catboost': cat_oof_prauc_cb}

    curr_cat_models = []
    for fold in range(5):
        model_cb = joblib.load(f"{category}_catboost_model_fold_{fold+1}.joblib")
        curr_cat_models.append(model_cb)

    ensemble_predictions_cb = predict_ensemble(test_category, curr_cat_models, features)
    joblib.dump(ensemble_predictions_cb, f"{category}_ensemble_predictions_cb.joblib")

    del ensemble_predictions_cb, model_cb, curr_cat_models
    gc.collect()

joblib.dump(cat_oof_preds_cb, "oof_preds_cb_5folds_cat1lvl.joblib")

category='Готовый бизнес и оборудование':   0%|          | 0/7 [00:00<?, ?it/s]

fold 1...
Learning rate set to 0.009044


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.4627636	test: 0.7385600	best: 0.7385600 (0)	total: 407ms	remaining: 5h 38m 52s
100:	learn: 0.5947639	test: 0.7856841	best: 0.7856841 (100)	total: 3.38s	remaining: 27m 50s
200:	learn: 0.6320305	test: 0.7888772	best: 0.7889414 (198)	total: 6.42s	remaining: 26m 30s
300:	learn: 0.6568757	test: 0.7911281	best: 0.7913776 (275)	total: 9.41s	remaining: 25m 54s
400:	learn: 0.6756496	test: 0.7912112	best: 0.7914934 (339)	total: 12.4s	remaining: 25m 29s
500:	learn: 0.6901884	test: 0.7908997	best: 0.7916216 (428)	total: 15.3s	remaining: 25m 11s
600:	learn: 0.7014128	test: 0.7907505	best: 0.7916216 (428)	total: 18.4s	remaining: 25m 8s
700:	learn: 0.7112560	test: 0.7917445	best: 0.7918474 (694)	total: 21.7s	remaining: 25m 23s
800:	learn: 0.7199480	test: 0.7926989	best: 0.7927471 (798)	total: 24.8s	remaining: 25m 25s
900:	learn: 0.7268824	test: 0.7932790	best: 0.7932828 (890)	total: 28s	remaining: 25m 26s
1000:	learn: 0.7329153	test: 0.7940448	best: 0.7940885 (996)	total: 31.1s	remaining:

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

100:	learn: 0.4612075	test: 0.4065386	best: 0.4065386 (100)	total: 5.22s	remaining: 43m
200:	learn: 0.4904948	test: 0.4108786	best: 0.4111138 (199)	total: 10.6s	remaining: 43m 43s
300:	learn: 0.5105297	test: 0.4076879	best: 0.4111138 (199)	total: 15.8s	remaining: 43m 35s
400:	learn: 0.5252225	test: 0.4069528	best: 0.4111138 (199)	total: 21.1s	remaining: 43m 27s
500:	learn: 0.5372766	test: 0.4065139	best: 0.4111138 (199)	total: 26.4s	remaining: 43m 28s
600:	learn: 0.5495169	test: 0.4049029	best: 0.4111138 (199)	total: 31.7s	remaining: 43m 28s
700:	learn: 0.5603737	test: 0.4040437	best: 0.4111138 (199)	total: 37s	remaining: 43m 24s
800:	learn: 0.5713882	test: 0.4022146	best: 0.4111138 (199)	total: 42.5s	remaining: 43m 32s
900:	learn: 0.5810877	test: 0.4013412	best: 0.4111138 (199)	total: 47.9s	remaining: 43m 31s
1000:	learn: 0.5912793	test: 0.4001764	best: 0.4111138 (199)	total: 53.2s	remaining: 43m 24s
1100:	learn: 0.6019987	test: 0.3988009	best: 0.4111138 (199)	total: 58.5s	remaining: 

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


100:	learn: 0.4562806	test: 0.3985692	best: 0.3985692 (100)	total: 5.4s	remaining: 44m 28s
200:	learn: 0.4888981	test: 0.4214360	best: 0.4214929 (199)	total: 10.6s	remaining: 43m 48s
300:	learn: 0.5113080	test: 0.4302917	best: 0.4302917 (300)	total: 15.9s	remaining: 43m 45s
400:	learn: 0.5259517	test: 0.4351537	best: 0.4353422 (396)	total: 21.2s	remaining: 43m 41s
500:	learn: 0.5392405	test: 0.4401704	best: 0.4401704 (500)	total: 26.5s	remaining: 43m 42s
600:	learn: 0.5516400	test: 0.4436260	best: 0.4438179 (596)	total: 31.9s	remaining: 43m 40s
700:	learn: 0.5646785	test: 0.4440407	best: 0.4440652 (690)	total: 37.5s	remaining: 43m 54s
800:	learn: 0.5757685	test: 0.4462745	best: 0.4464061 (799)	total: 42.8s	remaining: 43m 46s
900:	learn: 0.5871766	test: 0.4474502	best: 0.4476353 (889)	total: 48.1s	remaining: 43m 42s
1000:	learn: 0.5990742	test: 0.4486024	best: 0.4486884 (986)	total: 53.4s	remaining: 43m 35s
1100:	learn: 0.6102335	test: 0.4486025	best: 0.4490004 (1080)	total: 58.8s	remai


0it [00:00, ?it/s]
1it [00:02,  2.45s/it]
2it [00:04,  2.43s/it]
3it [00:07,  2.43s/it]
4it [00:09,  2.31s/it]
5it [00:11,  2.33s/it]
category='Животные': 100%|██████████| 7/7 [4:31:21<00:00, 2325.86s/it]


['oof_preds_cb_5folds_cat1lvl.joblib']

In [27]:
# # кб по категориям 2 уровня

# categories = train['unique_cat_2'].unique()
# categories = [cat.split('_')[0] for cat in categories if cat.split('_')[0] == cat.split('_')[1]]

# gkf = StratifiedGroupKFold(n_splits=5) # GroupKFold(n_splits=5)

# cat_oof_preds_cb = {}
# oof_metrics_cat = {}

# progress_bar = tqdm(categories)

# for category in progress_bar:
#     progress_bar.set_description(f"{category=}")

#     train_category = train[train['unique_cat_2'] == f'{category}_{category}']
#     test_category = test[test['unique_cat_2'] == f'{category}_{category}']

#     print(f'{len(train_category)=}, {len(test_category)=}')

#     if len(train_category) < 40000:
#         print('ээ ну маловато чет! соси')
#         continue
        
#     cat_oof_preds_cb[category] = np.zeros(len(train_category))

#     for fold, (train_idx, val_idx) in enumerate(gkf.split(train_category, train_category[target], groups=train_category['group_id'])):
#         print(f"fold {fold+1}...")
    
#         X_train_cat, X_val_cat = train_category.iloc[train_idx][features], train_category.iloc[val_idx][features]
#         y_train_cat, y_val_cat = train_category.iloc[train_idx][target], train_category.iloc[val_idx][target]

#         model_fold_cb = CatBoostClassifier(
#             iterations=50000,
#             # learning_rate=0.1,
#             cat_features=cat_features,
#             # auto_class_weights='Balanced',
#             eval_metric='PRAUC',
#             random_seed=3*42+fold,
#             early_stopping_rounds=1000,
#             task_type='GPU',
#             verbose=100
#         )

#         model_fold_cb.fit(X_train_cat, y_train_cat, eval_set=(X_val_cat, y_val_cat))
#         cat_oof_preds_cb[category][val_idx] = model_fold_cb.predict_proba(X_val_cat)[:, 1]

#         joblib.dump(model_fold_cb, f"{category}_catboost_model_fold_{fold+1}.joblib")

#         del model_fold_cb
#         gc.collect()

#     precision, recall, thresholds = precision_recall_curve(train_category[target], cat_oof_preds_cb[category])
#     cat_oof_prauc_cb = auc(recall, precision)
#     print(f'{cat_oof_prauc_cb=}')

#     oof_metrics_cat[category] = {'catboost': cat_oof_prauc_cb}

#     curr_cat_models = []
#     for fold in range(5):
#         model_cb = joblib.load(f"{category}_catboost_model_fold_{fold+1}.joblib")
#         curr_cat_models.append(model_cb)

#     ensemble_predictions_cb = predict_ensemble(test_category, curr_cat_models, features)
#     joblib.dump(ensemble_predictions_cb, f"{category}_ensemble_predictions_cb.joblib")

#     del ensemble_predictions_cb, model_cb, curr_cat_models
#     gc.collect()

# joblib.dump(cat_oof_preds_cb, "oof_preds_cb_5folds_cat2lvl.joblib")